Primero se hace la conexión con google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset'

In [ ]:
%cd '/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset'

/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset


In [ ]:
!ls

carladataset.zip  Dataset_vision  kaggle.json


Se descarga el dataset desde la página al drive

In [ ]:
!kaggle datasets download -d alechantson/carladataset

 92% 91.0M/98.4M [00:01<00:00, 75.7MB/s]
100% 98.4M/98.4M [00:01<00:00, 70.5MB/s]


In [ ]:
#!mkdir Dataset_vision
!mv carladataset.zip Dataset_vision

In [ ]:
%cd '/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset/Dataset_vision'


/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset/Dataset_vision


In [ ]:
!unzip carladataset.zip

Archive:  carladataset.zip
replace CARLA-DATASET.v1-carla_v1.coco/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Aquí se crean las carpetas con la estructura de un dataset YOLO, que es donde se guardarán las imágenes del dataset original tras su procesamiento

In [ ]:
#Crear la carpeta root
%cd '/content/gdrive/MyDrive/Vision_Maquina'
!mkdir train_data
%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data'
!mkdir images
%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data'
!mkdir labels
%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data/images'
!mkdir train
!mkdir val
%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data/labels'
!mkdir train
!mkdir val


%cd '/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset'

/content/gdrive/MyDrive/Vision_Maquina
/content/gdrive/MyDrive/Vision_Maquina/train_data
/content/gdrive/MyDrive/Vision_Maquina/train_data
/content/gdrive/MyDrive/Vision_Maquina/train_data/images
/content/gdrive/MyDrive/Vision_Maquina/train_data/labels
/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset


### Preproceso

In [ ]:
!pip install pycocotools

Se lee el archivo .json del dataset en formato COCO

In [ ]:
from pycocotools.coco import COCO

# Ruta al archivo COCO JSON de anotaciones del train
coco_annotation_file_train = '/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset/Dataset_vision/CARLA-DATASET.v1-carla_v1.coco/train/_annotations.coco.json'  # Ajusta la ruta según tu caso

# Cargar las anotaciones
coco = COCO(coco_annotation_file_train)


loading annotations into memory...
Done (t=0.50s)
creating index...
index created!


Se obtienen valores de interés de este archivo

In [ ]:
import numpy as np

# Obtener las categorías
categorias = coco.loadCats(coco.getCatIds())

# Obtener las imágenes
imagenes = coco.loadImgs(coco.getImgIds())

# Obtener las anotaciones
anotaciones = coco.loadAnns(coco.getAnnIds())

print(anotaciones)

[{'id': 0, 'image_id': 0, 'category_id': 7, 'bbox': [444, 199, 45, 43], 'area': 1935, 'segmentation': [], 'iscrowd': 0}, {'id': 1, 'image_id': 0, 'category_id': 10, 'bbox': [276, 233, 87, 85], 'area': 7395, 'segmentation': [], 'iscrowd': 0}, {'id': 2, 'image_id': 1, 'category_id': 7, 'bbox': [483, 189, 55, 53], 'area': 2915, 'segmentation': [], 'iscrowd': 0}, {'id': 3, 'image_id': 2, 'category_id': 6, 'bbox': [421, 185, 16, 35], 'area': 560, 'segmentation': [], 'iscrowd': 0}, {'id': 4, 'image_id': 2, 'category_id': 3, 'bbox': [207, 237, 19, 51], 'area': 969, 'segmentation': [], 'iscrowd': 0}, {'id': 5, 'image_id': 3, 'category_id': 4, 'bbox': [500, 141, 34, 71], 'area': 2414, 'segmentation': [], 'iscrowd': 0}, {'id': 6, 'image_id': 3, 'category_id': 1, 'bbox': [301, 245, 38, 67], 'area': 2546, 'segmentation': [], 'iscrowd': 0}, {'id': 7, 'image_id': 4, 'category_id': 7, 'bbox': [415, 206, 38, 35], 'area': 1330, 'segmentation': [], 'iscrowd': 0}, {'id': 8, 'image_id': 6, 'category_id': 

Se guardan las imágenes de entreno en listas localmente

In [ ]:
l_ruta_img=[]
for ruta in imagenes:
  ruta_imagen=ruta['file_name']
  l_ruta_img.append(ruta_imagen)

# Crear una lista para almacenar las imágenes
train = []
val = []

# Iterar sobre las imágenes de training y cargarlas en la lista

ruta='/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset/Dataset_vision/CARLA-DATASET.v1-carla_v1.coco/train/'

for imagen_filename in l_ruta_img:
    ruta_nueva=ruta+imagen_filename
    imagen = cv2.imread(ruta_nueva,0)
    train.append(imagen)
    img_train = np.array(train)


# Funciones para construcción del dataset

### Se extraen las características de la imagen (incluyendo labels) y se reescalan.

In [ ]:
def draw_boxes(image, boxes,he,wi):
    altura_imagen, ancho_imagen = image.shape
    x, y, w, h = boxes

    #COCO
    #x1 = int((x/wi) * ancho_imagen)
    #y1 = int((y/he) * altura_imagen)
    #x2 = int((w/wi) * ancho_imagen)
    #y2 = int((h/he) * altura_imagen)

    #YOLO NORMALIZADO
    x1 = float(x/wi)
    y1 = float(y/he)
    x2 = float(w/wi)
    y2 = float(h/he)

    return (x1+(x2/2)), (y1+(y2/2)), x2, y2

### Para los datos de entrenamiento, se recorre la lista de imágenes y el diccionario del .json para reorganizar las imágenes, eliminar las imágenes sin etiquetas.

También se guardan las imágenes en la carpeta creada en drive y se crean los .txt que almacenan los datos de los labels de cada imagen, estos archivos se guardan en drive.

In [ ]:
#lista para las caracteristicas de las imagenes
categorias_img=[]
bbox2=[]
l_ruta_img=[]
he2=[]
wi2=[]
img_id_lista=[]
Img_resize2_train = []
i=0

### Train data

%cd '/content/gdrive/MyDrive/Vision_Maquina'

#for que recorre las anotaciones y guarda, numero de imagen, categoria y los bbox
for ejemplo in anotaciones:
  img_id=ejemplo['image_id']
  categoria=ejemplo['category_id']
  lista_bbox=ejemplo['bbox']
  categorias_img.append(categoria)
  bbox2.append(lista_bbox)
  img_id_lista.append(img_id)

#for que recorre imagenes y guarda el nombre del archivo y su tamaño
for ruta in imagenes:
  ruta_imagen=ruta['file_name']
  largo=ruta['height']
  ancho=ruta['width']
  he2.append(largo)
  wi2.append(ancho)
  l_ruta_img.append(ruta_imagen)

#Guardar imagenes interpoladas al nuevo dataset
%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data/images/train'
for i in range(len(img_train)):
  if i in img_id_lista:
   img4=img_train[i]
   Img_resize2_train.append(cv2.resize(img4, (416,416), interpolation=cv2.INTER_NEAREST))
   cv2.imwrite(str(i)+".jpeg",cv2.resize(img4, (416,416), interpolation=cv2.INTER_NEAREST))

plt.imshow(Img_resize2_train[0])
#Guardar labels

%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data/labels/train'
for i in range(len(img_id_lista)):
  numero_imagen= img_id_lista[i]
  print(numero_imagen)
  bbox=bbox2[i]
  he=he2[numero_imagen]
  wi=wi2[numero_imagen]
  #Guardar imagen en nuevo dataset
  xi1,yi1,xi2,yi2 = draw_boxes(Img_resize2_train[numero_imagen],bbox,he,wi)

  if i!=0:
   if img_id_lista[i] != img_id_lista[i-1]:
      archn.close()
      del archn
  else:
    try:
      del archn
    except:
      pass

  try:
    print(archn)
  except NameError:
    archn=open(str(img_id_lista[i])+".txt","w")
    archn.write(str(categorias_img[i])+" "+str(xi1)+" "+str(yi1)+" "+str(xi2)+" "+str(yi2)+"\n")
  else:
    archn.write(str(categorias_img[i])+" "+str(xi1)+" "+str(yi1)+" "+str(xi2)+" "+str(yi2)+"\n")


### Para los datos de validación

Se realiza el mismo procedimiento que con los datos de train, naturalmente se cambia la ruta y se lee el .json correspondiente a las imágenes de validación

In [ ]:
# Ruta al archivo COCO JSON de anotaciones del valid
coco_annotation_file_val = '/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset/Dataset_vision/CARLA-DATASET.v1-carla_v1.coco/valid/_annotations.coco.json'  # Ajusta la ruta según tu caso

# Cargar las anotaciones
coco = COCO(coco_annotation_file_val)

# Obtener las categorías
categorias = coco.loadCats(coco.getCatIds())

# Obtener las imágenes
imagenes = coco.loadImgs(coco.getImgIds())

# Obtener las anotaciones
anotaciones = coco.loadAnns(coco.getAnnIds())

l_ruta_img=[]
for ruta in imagenes:
  ruta_imagen=ruta['file_name']
  l_ruta_img.append(ruta_imagen)

# Iterar sobre las imágenes de validación y cargarlas en la lista

ruta='/content/gdrive/MyDrive/Vision_Maquina/kaggle_dataset/Dataset_vision/CARLA-DATASET.v1-carla_v1.coco/valid/'

for imagen_filename in l_ruta_img:
    ruta_nueva=ruta+imagen_filename
    imagen = cv2.imread(ruta_nueva,0)
    val.append(imagen)
    img_val = np.array(val)


#lista para las caracteristicas de las imagenes
categorias_img=[]
bbox2=[]
l_ruta_img=[]
he2=[]
wi2=[]
img_id_lista=[]
Img_resize2_valid = []
i=0

### Validation data

%cd '/content/gdrive/MyDrive/Vision_Maquina'

#for que recorre las anotaciones y guarda, numero de imagen, categoria y los bbox
for ejemplo in anotaciones:
  img_id=ejemplo['image_id']
  categoria=ejemplo['category_id']
  lista_bbox=ejemplo['bbox']
  categorias_img.append(categoria)
  bbox2.append(lista_bbox)
  img_id_lista.append(img_id)

#for que recorre imagenes y guarda el nombre del archivo y su tamaño
for ruta in imagenes:
  ruta_imagen=ruta['file_name']
  largo=ruta['height']
  ancho=ruta['width']
  he2.append(largo)
  wi2.append(ancho)
  l_ruta_img.append(ruta_imagen)

#Guardar imagenes interpoladas al nuevo dataset
%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data/images/val'
for i in range(len(img_val)):
  if i in img_id_lista:
   img4=img_val[i]
   Img_resize2_valid.append(cv2.resize(img4, (416,416), interpolation=cv2.INTER_NEAREST))
   cv2.imwrite(str(i)+".jpeg",cv2.resize(img4, (416,416), interpolation=cv2.INTER_NEAREST))

plt.imshow(Img_resize2_valid[0])
#Guardar labels

%cd '/content/gdrive/MyDrive/Vision_Maquina/train_data/labels/val'
for i in range(len(img_id_lista)):

  numero_imagen= img_id_lista[i]
  print(numero_imagen)
  bbox=bbox2[i]
  he=he2[numero_imagen]
  wi=wi2[numero_imagen]
  #Guardar imagen en nuevo dataset
  xi1,yi1,xi2,yi2 = draw_boxes(Img_resize2_valid[numero_imagen],bbox,he,wi)

  if i!=0:
   if img_id_lista[i] != img_id_lista[i-1]:
      archn.close()
      del archn
   else:
     try:
       del archn
     except:
       pass
  try:
    print(archn)
  except NameError:
    archn=open(str(img_id_lista[i])+".txt","w")
    archn.write(str(categorias_img[i])+" "+str(xi1)+" "+str(yi1)+" "+str(xi2)+" "+str(yi2)+"\n")
  else:
    archn.write(str(categorias_img[i])+" "+str(xi1)+" "+str(yi1)+" "+str(xi2)+" "+str(yi2)+"\n")


# El dataset queda almacenado en la carpeta de drive